<a href="https://colab.research.google.com/github/jwiggerthale/HiL-Machine-Learnig/blob/main/Datasplitforyolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import kagglehub
from glob import glob
import os
import yaml

In [ ]:
dataset_path = kagglehub.dataset_download("zhangyunsheng/defects-class-and-location")

label_files = glob(os.path.join(dataset_path, '**/*.xml'), recursive=True)
image_files = glob(os.path.join(dataset_path, '**/images/*/*.jpg'), recursive=True)  # Adjusted pattern
label_files[:5], image_files[:5]

(['/kaggle/input/defects-class-and-location/label/label/img_01_425241700_00384.xml',
  '/kaggle/input/defects-class-and-location/label/label/img_06_425505000_00050.xml',
  '/kaggle/input/defects-class-and-location/label/label/img_06_3436814300_00056.xml',
  '/kaggle/input/defects-class-and-location/label/label/img_07_425390900_00077.xml',
  '/kaggle/input/defects-class-and-location/label/label/img_01_425005700_00501.xml'],
 ['/kaggle/input/defects-class-and-location/images/images/inclusion/img_05_425243500_00898.jpg',
  '/kaggle/input/defects-class-and-location/images/images/inclusion/img_03_3402617100_00131.jpg',
  '/kaggle/input/defects-class-and-location/images/images/inclusion/img_06_425508200_00471.jpg',
  '/kaggle/input/defects-class-and-location/images/images/inclusion/img_08_425506100_01068.jpg',
  '/kaggle/input/defects-class-and-location/images/images/inclusion/img_03_3402617100_00120.jpg'])

In [ ]:
# from PIL import Image
# im = Image.open('/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/images/images/waist folding/img_08_425501100_00051.jpg')
# print(im.size)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf /content/datanew1

In [ ]:
class_map = {
    'crease': 0, 'crescent_gap': 1, 'inclusion': 2,
    'oil_spot': 3, 'punching_hole': 4, 'rolled_pit': 5,
    'silk_spot': 6, 'waist_folding': 7, 'water_spot': 8,
    'welding_line': 9
}



In [ ]:
# import os

# label_folder = '/content/drive/MyDrive/ims'

# for filename in os.listdir(label_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(label_folder, filename)
#         with open(file_path, 'r') as f:
#             lines = f.readlines()
#         new_lines = []
#         for line in lines:
#             parts = line.strip().split()
#             if parts:
#                 class_name = parts[0]
#                 if class_name in class_map:
#                     parts[0] = str(class_map[class_name])
#                 new_lines.append(' '.join(parts))
#         with open(file_path, 'w') as f:
#             f.write('\n'.join(new_lines))
# print('Conversion complete.')

In [ ]:
# import os

# label_folder = '/content/drive/MyDrive/ims'

# for filename in os.listdir(label_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(label_folder, filename)
#         with open(file_path, 'r') as f:
#             lines = f.readlines()
#         new_lines = []
#         found_folding = False
#         for line in lines:
#             parts = line.strip().split()
#             if 'folding' in parts:
#                 found_folding = True
#                 parts = [p for p in parts if p != 'folding']
#             new_lines.append(' '.join(parts))
#         if found_folding:
#             with open(file_path, 'w') as f:
#                 f.write('\n'.join(new_lines))
#             print(f"Updated: {filename}")

In [ ]:
import os
import shutil
import random
from pathlib import Path

# Set your  directories
images_dir = Path('/kaggle/input/defects-class-and-location/images/images')
labels_dir = Path('/content/drive/MyDrive/ims')

# Set output directories
output_dir = Path('datanew1')
train_dir = output_dir / 'train'
val_dir = output_dir / 'val'

# Split ratio
split_ratio = 0.6

# Gather label files
label_files = sorted([f for f in labels_dir.iterdir() if f.is_file()])

# Shuffle and split
random.shuffle(label_files)
split_idx = int(len(label_files) * split_ratio)
train_labels = label_files[:split_idx]
val_labels = label_files[split_idx:]

def copy_data(label_list, split_folder):
    (split_folder / 'labels').mkdir(parents=True, exist_ok=True)
    (split_folder / 'images').mkdir(parents=True, exist_ok=True)
    for label_path in label_list:
        shutil.copy(label_path, split_folder / 'labels' / label_path.name)
        base_name = label_path.stem
        found = False
        for class_folder in images_dir.iterdir():
            if class_folder.is_dir():
                image_path = class_folder / f"{base_name}.jpg"
                if image_path.exists():
                    shutil.copy(image_path, split_folder / 'images' / image_path.name)
                    found = True
                    break
        if not found:
            print(f"Image for label {label_path.name} not found.")

copy_data(train_labels, train_dir)
copy_data(val_labels, val_dir)


In [ ]:
import os
import shutil

# Define source directories
train_dir = 'train'
val_dir = 'val'

# Define destination directory
all_images_dir = 'all_images'

# Create destination directory if it doesn't exist
os.makedirs(all_images_dir, exist_ok=True)

# Supported image extensions
image_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.gif'}

# Function to copy images from source to destination with renaming duplicates
def copy_images(src_dir, dest_dir):
    for root, _, files in os.walk(src_dir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            if ext in image_extensions:
                src_path = os.path.join(root, file)
                dest_path = os.path.join(dest_dir, file)
                # Rename if file exists
                count = 1
                while os.path.exists(dest_path):
                    name, ext = os.path.splitext(file)
                    new_name = f"{name}_{count}{ext}"
                    dest_path = os.path.join(dest_dir, new_name)
                    count += 1
                shutil.copy2(src_path, dest_path)

# Copy images from train and val folders
copy_images(train_dir, all_images_dir)
copy_images(val_dir, all_images_dir)

print('All images have been copied to the folder:', all_images_dir)


In [ ]:
def create_data_yaml(train_dir, val_dir, class_map, save_path='data.yaml'):
    """Creates a data.yaml file for YOLO training."""
    data = {
        'train': train_dir,
        'val': val_dir,
        'nc': len(class_map),
        'names': list(class_map.keys())
    }
    with open(save_path, 'w') as f:
        yaml.dump(data, f)
    print(f"data.yaml created at {save_path}")



create_data_yaml(
    train_dir='/content/datanew1/train',
    val_dir='/content/datanew1/val',
    class_map=class_map,
    save_path='datanew1.yaml'
)

data.yaml created at datanew1.yaml


In [ ]:
!pip install ultralytics==8.3.155
!pip install ultralytics-thop==2.0.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import torch

from ultralytics import YOLO


class YOLOv11Trainer:

    def __init__(self, model_path, data_yaml, device):

        self.device = 'cpu'# 'cuda:0'#device if torch.cuda.is_available() else "cpu"

        self.model = YOLO(model_path)

        self.data_yaml = data_yaml

    def train(self, epochs=1, imgsz=480):

        train_results = self.model.train(

            data=self.data_yaml,

            epochs=epochs,

            imgsz=imgsz,

            device=self.device

        )

        return self.model



trainer = YOLOv11Trainer(model_path="yolo11n.pt", data_yaml=f"datanew1.yaml", device='cpu')

model = trainer.train(epochs=10, imgsz=[640,312])

torch.save(model.state_dict('Tuned_YOLO11n.pth'))

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 57.4MB/s]


New https://pypi.org/project/ultralytics/8.3.156 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datanew1.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=[640, 312], int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer

100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]

Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

...

                   all        912        912      0.601       0.52      0.519      0.266



10 epochs completed in 3.798 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [02:22<00:00,  4.91s/it]


                   all        912        912      0.601       0.52      0.519      0.266
                crease         16         16      0.315     0.0625     0.0697     0.0355
          crescent_gap         84         84      0.766      0.917      0.867       0.55
             inclusion         92         92      0.288       0.12      0.193      0.076
              oil_spot         95         95      0.476      0.474      0.469      0.212
         punching_hole        100        100      0.719       0.95      0.882      0.511
            rolled_pit         13         13          1          0     0.0975     0.0461
             silk_spot        243        243      0.616      0.519      0.553      0.221
         waist_folding         57         57      0.443      0.561      0.513      0.229
            water_spot        109        109      0.812      0.872      0.891       0.49
          welding_line        103        103      0.575      0.728      0.652      0.291
Speed: 1.3ms preproce

TypeError: 'str' object does not support item assignment

In [ ]:

!cp /content/runs/detect/train/weights/best.pt /content/drive/MyDrive/

In [ ]:
from google.colab import files
files.download('/content/runs/detect/train/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp /content/datanew1 /content/drive/MyDrive/

cp: -r not specified; omitting directory '/content/datanew1'
